In [ ]:
'''
Оптимизация c помощью optuna

Оптимизируемые параметры:
lr - learning rate для optim.Adam
n_filters1, n_filters2 - число фильтров для первого и второго сверточного слоя
batch_sie - размер батчей для обучающих данных

'''



In [5]:
import optuna
from torch.utils.data import random_split
import torch.optim as optim

In [ ]:
# Import previous files
%run data_preparation.ipynb
%run model.ipynb

In [2]:
train_losses = []
val_losses = []

In [6]:
# Optuna
def objective(trial):
    # Гиперпараметры, которые подбираем
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    n_filters1 = trial.suggest_int('n_filters1', 8, 32, step=8)
    n_filters2 = trial.suggest_int('n_filters2', 16, 64, step=16)
    
    # Обучающие и валидационные данные
    full_dataset = MNIST(root='./data', train=True, download=True, transform=trasform)
    train_size = int(0.8 * len(full_dataset))
    val_size = len(full_dataset) - train_size
    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=1000, shuffle=False)
    
    # Модель
    model = CNN(n_filters1, n_filters2)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr = lr)
    
    # Обучение (5 эпох)
    for epoch in range(5):
        # Обучение
        model.train()
        running_train_loss = 0
        for iter, (X_batch, y_batch) in enumerate(train_loader):
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            running_train_loss += loss.item() * X_batch.size(0)
        avg_train_loss = running_train_loss / len(train_loader.dataset)
        train_losses.append(avg_train_loss)
        # Валидация
        model.eval()
        running_val_loss = 0
        with torch.no_grad():
            for iter, (X_batch, y_batch) in enumerate(val_loader):
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                running_val_loss += loss.item() * X_batch.size(0)
        avg_val_loss = running_val_loss / len(val_loader.dataset)
        val_losses.append(avg_val_loss)
        return avg_val_loss

In [7]:
# Запуск Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20, timeout=500)

[I 2025-06-09 16:53:25,287] A new study created in memory with name: no-name-16514037-705c-4236-aabe-256184774136
[I 2025-06-09 16:53:37,653] Trial 0 finished with value: 0.06775864617278178 and parameters: {'batch_size': 128, 'lr': 0.007331318053393051, 'n_filters1': 16, 'n_filters2': 64}. Best is trial 0 with value: 0.06775864617278178.
[I 2025-06-09 16:53:46,739] Trial 1 finished with value: 0.19741836562752724 and parameters: {'batch_size': 64, 'lr': 0.00021154427148648255, 'n_filters1': 16, 'n_filters2': 16}. Best is trial 0 with value: 0.06775864617278178.
[I 2025-06-09 16:53:58,893] Trial 2 finished with value: 0.1658902478714784 and parameters: {'batch_size': 32, 'lr': 0.00018190183981776074, 'n_filters1': 24, 'n_filters2': 16}. Best is trial 0 with value: 0.06775864617278178.
[I 2025-06-09 16:54:12,111] Trial 3 finished with value: 0.0639663211380442 and parameters: {'batch_size': 64, 'lr': 0.002566843804311677, 'n_filters1': 32, 'n_filters2': 16}. Best is trial 3 with value: 

In [9]:
print("Best hyperparams: ", study.best_params)
print(f"Min loss: {study.best_value:.2%}")

Best hyperparams:  {'batch_size': 32, 'lr': 0.0032714662050235, 'n_filters1': 8, 'n_filters2': 64}
Min loss: 5.84%
